In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import matplotlib as mpl
import seaborn as sns
import scipy as sp
from sklearn import metrics

<br>
#### Genetic Algorithm

In [2]:
class genetic_algorithm:
    def __init__(self):
        pass
    
    def evaluate(self, x, metric='log10t'):
        # convert to array
        x = np.array(x)
        if metric == 'log10t': # 참가자 소지금 min max 차이가 ln(10 thousand dollars) 일때 1
            score = np.log(np.max(x) - np.min(x)) / np.log(100000)
            return score
    
    def crossover(self, character0, character1):
        gene0 = character0.genotype
        gene1 = character1.genotype
        
        # locus 0,1 crossover
        buff = gene0[[0,1]]
        gene0[[0,1]] = gene1[[0,1]]
        gene1[[0,1]] = buff
        
        # locus 5,6 crossover
        buff = gene0[[5,6]]
        gene0[[5,6]] = gene1[[5,6]]
        gene1[[5,6]] = buff
        
        # locus 10,11 crossover
        buff = gene0[[10,11]]
        gene0[[10,11]] = gene1[[10,11]]
        gene1[[10,11]] = buff
        
        return gene0, gene1
    
    def mutation(self, gene, rate=0.1):
        rv = sp.stats.bernoulli(0.1)
        mask = rv.rvs(10)
        # mutate
        for i in range(len(mask)):
            if mask[i] == 1:
                if gene[i] == 0:
                    gene[i] = 1
                elif gene[i] == 1:
                    gene[i] = 0
        return gene
        
        
    def genetic_similarity(gene0, gene1):
        if gene0.shape == gene1.shape:
            length = len(gene0)
            monitor = np.zeros(length)
            monitor[gene0 == gene1] = 1
            identical_num = np.sum(monitor)
            return(identical_num / length)
        
    def phenotype_variance(generation):
        var_imp = np.var(generation[:,0]) # 충동성 표현형 분산
        var_str = np.var(generation[:,1]) # 스트레스 저항 표현형 분산
        return(var_imp, var_str, var_imp + var_str)

In [3]:
ga = genetic_algorithm()

<br>
#### Blackjack Game

character

1 +0.4  
1 +0.3  
1 +0.2  
1 +0.1  
1 *0.5  

In [4]:
class character:
    def __init__(self, pheno_noise=.1):
        self.pheno_noise = pheno_noise
        self.genotype = np.random.randint(low=0, high=2, size=15)
        self.phenotype = self.represent_phenotype()
        
        
    def represent_phenotype(self):
        phenotype = {}
        keys = ['impulsivity', 'stress_reg', 'thresh_hit_proba']
        for key in keys:
            if key == 'impulsivity': # decode impulsivity
                pheno = self.genotype[0] * 0.4 + self.genotype[1] * 0.3 + self.genotype[2] * 0.2 + self.genotype[3] * 0.1
                pheno -= pheno * self.genotype[4] * 0.5
            elif key == 'stress_reg': # decode stress_reg
                pheno = self.genotype[5] * 0.4 + self.genotype[6] * 0.3 + self.genotype[7] * 0.2 + self.genotype[8] * 0.1
                pheno -= pheno * self.genotype[9] * 0.5
            elif key == 'thresh_hit_proba':
                pheno = self.genotype[10] * 0.4 + self.genotype[11] * 0.3 + self.genotype[12] * 0.2 + self.genotype[13] * 0.1
                pheno -= pheno * self.genotype[14] * 0.5
            pheno += np.random.normal(loc=0, scale=self.pheno_noise)
            phenotype.update({key:pheno})
                
        return phenotype

<br>
#### blackjack 게임 세팅

블랙잭 점수 계산

In [5]:
def calc_score(card, minmax='min'):
    score = 0
    
    if card[1] in ['K', 'Q', 'J']:
        score = 10
    elif card[1] == 'A':
        if minmax == 'min':
            score = 1
        elif minmax == 'max':
            score = 10
    else:
        score = int(card[1])
    return score

<br>
정서 함수

In [6]:
def calc_emotion(player, mode, pleasure_step=1):
    if len(player.bet_path) != 1:
        arousal = np.tanh((1 - player.character.phenotype['stress_reg']) * 
                          ((player.bet_path[-1] - player.bet_path[-2]) / float(player.bet_path[-2])))
    elif len(player.bet_path) == 1:
        arousal = np.tanh((1 - player.character.phenotype['stress_reg']))
    
    if mode == 'win':
        pleasure = np.tanh((1 - player.character.phenotype['stress_reg']) * np.e ** (player.bet / player.budget))
    elif mode == 'lose':
        if player.budget > 0:
            pleasure = - np.tanh((1 - player.character.phenotype['stress_reg']) * np.e ** (player.bet / player.budget))
        else:
            pleasure = - np.tanh((1 - player.character.phenotype['stress_reg']) * np.e)
    
    return arousal, pleasure

<br>
배팅 금액 함수

In [7]:
def calc_bet(player):
    bet = player.bet + (player.bet * np.e ** (player.arousal + player.character.phenotype['impulsivity']) * player.pleasure)
    return round(bet)

<br>
플레이어 세팅

In [81]:
class player:
    def __init__(self, character, budget, bet):
        self.budget = budget # 소지금
        self.bet = bet # 베팅 금액
        self.character = character
        
        # emotion
        self.arousal = 0
        self.pleasure = 0
        
        # game values
        self.card = []
        self.min_score = 0
        self.max_score = 0
        
        # status
        self.status = 'ongame'
        
        # thresh_hit_proba(player가 stay할 때 질 확률, 커질 수록 보수적인 hit: 반드시 질 것 같을 때만 hit)
        self.thresh_hit_proba = self.character.phenotype['thresh_hit_proba']
        
        # winlose
        self.game_result = ''
        
        # log
        self.budget_path = [self.budget]
        self.bet_path = [self.bet]
        self.arousal_path = [self.arousal]
        self.pleasure_path = [self.pleasure]
        self.thresh_hit_proba_path = [self.thresh_hit_proba]
        
    def betting(self):
        return self.bet
    
    def evaluate(self, visible_dealer_score):
        if visible_dealer_score != 7:
            dealer_score_range = np.arange(17, visible_dealer_score + 10) # 딜러의 hidden card가 open될 때 가능한 score 범위
            dealer_safe_range = dealer_score_range[dealer_score_range <= 21] # 딜러가 21 이하의 수가 나올 범위
            dealer_burst_range = dealer_score_range[dealer_score_range > 21] # 딜러가 burst될 범위
            proba_dealer_burst = len(dealer_burst_range) / float(len(dealer_score_range)) # dealer가 burst 되어 stay해도 이길 확률
        else:
            proba_dealer_burst = 0
        if self.status == 'burst':
            return self.stay()
        elif self.status == 'blackjack':
            return self.stay()
        elif self.status == 'ongame':
            if self.max_score <= 11: # 최대 스코어가 11이하면 burst 위험 없으므로 무조건 hit
                return self.hit()
            else:
                player_score_range = np.arange(self.min_score + 1, self.min_score + 10) # player가 hit 했을때 가능한 score 범위
                player_safe_range = player_score_range[player_score_range <= 21] # hit 했을 때 player가 21이하일 범위
                player_burst_range = player_score_range[player_score_range > 21] # hit 했을 때 player가 burst될 범위
                proba_player_burst = len(player_burst_range) / float(len(player_score_range)) # player가 burst되어 hit하면 질 확률
                
                # stay 했을 때 질 확률
                if self.min_score <= 16:
                    proba_stay_lose = (1 - proba_dealer_burst)
                elif self.min_score >= 17:
                    proba_stay_lose = (1 - proba_dealer_burst) * ((21 - self.min_score) / 5.0)
                
                # hit/ stay 선택
                if proba_stay_lose >= self.thresh_hit_proba:
                    return self.hit()
                else:
                    return self.stay()
    
    def hit(self):
        return 'hit'
    
    def stay(self):
        return 'stay'
    
    def update_score(self):
        self.min_score = np.sum(np.array([calc_score(x, minmax='min') for x in self.card]))
        self.max_score = np.sum(np.array([calc_score(x, minmax='max') for x in self.card]))
        if (self.min_score == 21) | (self.max_score == 21):
            self.status = 'blackjack'
        elif self.min_score > 21:
            self.status = 'burst'
    
    def winlose(self):
        if self.game_result == 'win':
            # 승패에 따라 hit proba 조정, 이기면 공격적인 hit
            self.thresh_hit_proba -= (self.bet / self.budget) * self.thresh_hit_proba * \
                                     (0.5 + self.character.phenotype['impulsivity'])
            if self.status == 'blackjack':
                self.budget += (self.bet) * 1.5
            else:
                self.budget += self.bet
            self.arousal, self.pleasure = calc_emotion(self, 'win')
        elif self.game_result == 'lose':
            # 승패에 따라 hit proba 조정, 지면 보수적인 hit
            self.thresh_hit_proba += (self.bet / self.budget) * self.thresh_hit_proba * \
                                     (0.5 + self.character.phenotype['impulsivity'])            
            self.budget -= self.bet
            # 0보다 작은 budget 0으로 보정
            if self.budget < 0:
                self.budget = 0
            
            self.arousal, self.pleasure = calc_emotion(self, 'lose')
        elif self.game_result == 'even':
            pass

        self.bet = calc_bet(self)
        if self.bet > self.budget:
            self.bet = self.budget
        self.bet_path.append(self.bet)
        self.budget_path.append(self.budget)
        self.arousal_path.append(self.arousal)
        self.pleasure_path.append(self.pleasure)
        self.thresh_hit_proba_path.append(self.thresh_hit_proba)

<br>
딜러 세팅

In [82]:
class dealer:
    def __init__(self):
        # game values
        self.card = []
        self.score = 0
        self.hidden_card = []
        self.visible_card = []
        
        # status
        self.status = 'ongame'
    
    def update_score(self):
        self.hidden_card = self.card[-1] # 딜러가 받은 마지막 장은 hidden
        self.visible_card = self.card[:-1] # 나머지는 open
        self.score = np.sum(np.array([calc_score(x, minmax='min') for x in self.card])) # dealer의 합계 점수
        self.visible_score = np.sum(np.array([calc_score(x, minmax='min') for x in self.visible_card])) # player에게 보여지는 점수
        if self.score == 21:
            self.status = 'blackjack'
        elif self.score > 21:
            self.status = 'burst'

<br>
게임 세팅

In [83]:
class blackjack:
    def __init__(self, dealer, player, card):
        self.dealer = dealer
        self.player = player
        self.card = card
        self.player_score_path = []
        self.player_card_path = []
        self.dealer_score_path = []
        self.dealer_card_path = []
        self.game_result_path = []
    
    def set_bet_money(self, money):
        self.money = money

    def pick_card(self):
        if len(self.card) != 0:
            pass
        else:
            mark = ['H', 'C', 'S', 'D']
            number = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
            self.card = [(i, j) for i in mark for j in number]
        return self.card[np.random.randint(0, len(self.card))]
    
    def draw(self):
        
        # dealer
        while (self.dealer.score < 17):
            pick = self.pick_card()
            self.card.remove(pick)
            self.dealer.card.append(pick)
            self.dealer.update_score()
        
        # player
        for i in range(2):
            pick = self.pick_card()
            self.card.remove(pick)
            self.player.card.append(pick)
            self.player.update_score()
    
    def hit(self):
        pick = self.pick_card()
        self.card.remove(pick)
        self.player.card.append(pick)
        self.player.update_score()

<br>
게임 플레이

In [84]:
def single_game(game):
    game.player.card = []
    game.player.score = 0
    game.player.status = 'ongame'
    game.dealer.card = []
    game.dealer.score = 0
    game.dealer.status = 'ongame'
    
    game.draw()
    while(game.player.evaluate(game.dealer.visible_score) != 'stay'):
        game.hit()
    
    # player의 유효 점수 계산 (21 넘지 않는 가장 큰 수)
    if game.player.max_score > 21:
        valid_score = game.player.min_score
    else:
        valid_score = game.player.max_score
        
    # 승/패 판정
    if game.player.status == 'burst': # player가 burst면 무조건 패배
        game.player.game_result = 'lose'
    else:
        if game.dealer.status == 'burst': # dealer가 burst면 무조건 승리
            game.player.game_result = 'win'
        elif valid_score > game.dealer.score: 
            game.player.game_result = 'win'
        elif valid_score < game.dealer.score:
            game.player.game_result = 'lose'
        elif valid_score == game.dealer.score:
            game.player.game_result = 'even'
    
    # 결과 반영
    game.player.winlose()
    
    # 결과 기록
    game.player_score_path.append(valid_score)
    game.player_card_path.append(game.player.card)
    game.dealer_score_path.append(game.dealer.score)
    game.dealer_card_path.append(game.dealer.card)
    game.game_result_path.append(game.player.game_result)

In [85]:
def simulator(game, MAX_ROUND=500):
    cnt = 0
    while ((cnt < MAX_ROUND) & (game.player.budget > 0) & (game.player.bet > 0)) | (game.player.budget >= 100000):
        single_game(game)
        cnt += 1

In [86]:
def single_generation(bj_games):
    for x in bj_games:
        simulator(x)
    return bj_games

In [87]:
def select_survivals(bj_games, how, proba_norm=0, rank_norm=.5):
    survival_proba = [ga.evaluate(x.player.budget_path) for x in bj_games]
    for i in range(len(survival_proba)):
        if survival_proba[i] > 1:
            survival_proba[i] = 1
    if how == 'proba':
        temp = pd.DataFrame(survival_proba, columns=['proba'])
        temp['select_mask'] = [np.random.choice([False, True], size=1, p=[1-x, x])[0] for x in survival_proba]
        temp.select_mask[temp.proba < proba_norm] = False
        select_mask = temp.select_mask.values
    elif how == 'rank':
        temp = pd.DataFrame(survival_proba, columns=['proba'])
        select_mask = temp.sort_values(by='proba', ascending=False).index.values[:round(len(temp) * rank_norm)]
    survivals = bj_games[select_mask]
    return survival_proba, survivals

In [88]:
def next_generation(survivals, mate_round=1, max_pop=200):
    next_genotype = []
    for mate in range(mate_round):
        np.random.shuffle(survivals)
        for i in range(len(survivals) - 1):
            siblings = ga.crossover(survivals[i].player.character, survivals[i+1].player.character)
            next_genotype += list(siblings)
    next_genotype = np.array([ga.mutation(x) for x in next_genotype])
    if len(next_genotype) > max_pop:
        next_genotype = next_genotype[:200]
    return next_genotype

<br>
실험 초기화

In [179]:
game_path = []
survial_proba_path = []
survivals_path = []
round_cnt_path = []

# player 초기화
players = np.array([player(budget=500, bet=10, character=character()) for i in range(100)])
# 카드 초기값
mark = ['H', 'C', 'S', 'D']
number = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
card = [(i, j) for i in mark for j in number]
bj_games = np.array([blackjack(dealer=dealer(), player=x, card=card) for x in players]) 

<br>
세대 진행

In [180]:
for round_cnt in range(100):
    game_path.append(single_generation(bj_games))
    proba_temp, survivals = select_survivals(game_path[-1], how='proba', proba_norm=.25, rank_norm=.2)
    survial_proba_path.append(proba_temp)
    survivals_path.append(survivals)
    round_cnt_path.append([len(x.game_result_path) for x in game_path[-1]])
    # cross-over and mutate for next generation
    next_genotype = next_generation(survivals, mate_round=3, max_pop=300)  
    # player 초기화
    players = np.array([player(budget=500, bet=10, character=character()) for i in range(len(next_genotype))])
    # next_genotype 입력
    for i in range(len(players)):
        players[i].character.genotype = next_genotype[i]
        players[i].character.phenotype = players[i].character.represent_phenotype()
    # 카드 초기값
    mark = ['H', 'C', 'S', 'D']
    number = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    card = [(i, j) for i in mark for j in number]
    bj_games = np.array([blackjack(dealer=dealer(), player=x, card=card) for x in players]) 
    print('세대 수: {}  개체 수: {}  평균적합도: {}  생존: {}  round: {}'.format(len(game_path), len(game_path[-1]),
                                                np.round(np.mean(survial_proba_path[-1]), 3), len(survivals_path[-1]),
                                                np.round(np.mean(round_cnt_path[-1]), 3)))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


세대 수: 1  개체 수: 100  평균적합도: 0.288  생존: 17  round: 75.45
세대 수: 2  개체 수: 96  평균적합도: 0.319  생존: 15  round: 100.375
세대 수: 3  개체 수: 84  평균적합도: 0.302  생존: 18  round: 102.238
세대 수: 4  개체 수: 102  평균적합도: 0.298  생존: 18  round: 134.147
세대 수: 5  개체 수: 102  평균적합도: 0.294  생존: 21  round: 73.765
세대 수: 6  개체 수: 120  평균적합도: 0.356  생존: 30  round: 186.475
세대 수: 7  개체 수: 174  평균적합도: 0.302  생존: 41  round: 122.897
세대 수: 8  개체 수: 240  평균적합도: 0.309  생존: 53  round: 118.071
세대 수: 9  개체 수: 200  평균적합도: 0.311  생존: 36  round: 133.3
세대 수: 10  개체 수: 210  평균적합도: 0.311  생존: 42  round: 116.671
세대 수: 11  개체 수: 246  평균적합도: 0.299  생존: 48  round: 104.341
세대 수: 12  개체 수: 282  평균적합도: 0.311  생존: 68  round: 131.113
세대 수: 13  개체 수: 200  평균적합도: 0.289  생존: 43  round: 100.515
세대 수: 14  개체 수: 252  평균적합도: 0.292  생존: 46  round: 79.754
세대 수: 15  개체 수: 270  평균적합도: 0.304  생존: 62  round: 125.011
세대 수: 16  개체 수: 200  평균적합도: 0.29  생존: 37  round: 80.08
세대 수: 17  개체 수: 216  평균적합도: 0.307  생존: 44  round: 134.847
세대 수: 18  개체 수: 258  평균적합도: 0.305 

<br>
log selection

In [181]:
idx_i = []
idx_j = []
threshold = .6
for i in range(len(survial_proba_path)):
    for j in range(len(survial_proba_path[i])):
        if survial_proba_path[i][j] >= threshold:
            idx_i.append(i)
            idx_j.append(j)

In [182]:
game_cand = []
for i, j in zip(idx_i, idx_j):
    game_cand.append(game_path[i][j])
len(game_cand)

228

<br>
generate story

In [188]:
def write_story(game):
    for i in range(len(game.player.bet_path)-1):
        print('{} 번째 게임'.format(i+1))
        
        if i == 0:
            print('가진 돈은 500$')
            print('우선 {}$를 베팅해본다'.format(game.player.bet_path[i]))
        else:
            if game.player.bet_path[i] > game.player.bet_path[i-1]:
                adv = '베팅 금액을 올려서'
            elif game.player.bet_path[i] < game.player.bet_path[i-1]:
                adv = '베팅 금액을 내려서'
            else:
                adv = '이번에도'
            if game.player.bet_path[i] == game.player.budget_path[i]:
                print('{}$ 전부 걸어 보자!'.format(game.player.bet_path[i]))
            else:
                print('{} 현재 가진 돈 {}$ 중 {}$를 베팅한다'.format(adv, game.player.budget_path[i], game.player.bet_path[i]))
        if i < len(game.player.bet_path)-1:
            player_card_init = game.player_card_path[i][:2]
            print('내가 받은 카드는 {}'.format([''.join(x for x in card) for card in player_card_init]))
            player_score_init = np.sum(calc_score(x) for x in player_card_init)
            print('{} 점'.format(player_score_init))
            dealer_card_init = game.dealer_card_path[i][:-1]
            print('딜러의 카드는 {}와 한장의 히든 카드'.format([''.join(x for x in card) for card in dealer_card_init]))
            print('보이는 점수는 {} 점'.format(np.sum(calc_score(x) for x in dealer_card_init)))
        if game.player.thresh_hit_proba_path[i] >= 0.7:
            print("'STAY 하는게 안전할 것 같은데... ({})'".format(round(game.player.thresh_hit_proba_path[i], 3)))
        elif game.player.thresh_hit_proba_path[i] < 0.3:
            print("'HIT 해도 괜찮을 것 같은데! ({})'".format(round(game.player.thresh_hit_proba_path[i], 3)))       
        
        # Hit/ Stay log
        if len(game.player_card_path[i]) == 2:
            print('STAY!')
        else:
            for card in game.player_card_path[i][2:]:
                print('HIT!')
                print([''.join(x for x in card)])
        
        # result
        print('player: {}'.format(game.player_card_path[i]))
        print('{}'.format(game.player_score_path[i]))
        print('dealer: {}'.format(game.dealer_card_path[i]))
        print('{}'.format(game.dealer_score_path[i]))
        if game.game_result_path[i] == 'win':
            if game.player_score_path[i] == 21:
                print('Blackjack!')
            else:
                print('이겼다!')
        elif game.game_result_path[i] == 'even':
            print('비겼다')
        elif game.game_result_path[i] == 'lose':
            print('졌다')
            if game.player.budget_path[i+1] == 0:
                print('가진 돈 전부를 잃었다...')
        print('arousal:{}  pleasure{}'.format(round(game.player.arousal_path[i+1], 3),
                                              round(game.player.pleasure_path[i+1], 3)))
        
        print('')

In [194]:
write_story(game_cand[2])

1 번째 게임
가진 돈은 500$
우선 10$를 베팅해본다
내가 받은 카드는 ['S7', 'HJ']
17 점
딜러의 카드는 ['DA', 'S2', 'HA', 'SA', 'CK']와 한장의 히든 카드
보이는 점수는 15 점
STAY!
player: [('S', '7'), ('H', 'J')]
17
dealer: [('D', 'A'), ('S', '2'), ('H', 'A'), ('S', 'A'), ('C', 'K'), ('C', '7')]
22
이겼다!
arousal:0.666  pleasure0.674

2 번째 게임
베팅 금액을 올려서 현재 가진 돈 510$ 중 34.0$를 베팅한다
내가 받은 카드는 ['SJ', 'D9']
19 점
딜러의 카드는 ['H7']와 한장의 히든 카드
보이는 점수는 7 점
STAY!
player: [('S', 'J'), ('D', '9')]
19
dealer: [('H', '7'), ('C', 'J')]
17
이겼다!
arousal:0.958  pleasure0.693

3 번째 게임
베팅 금액을 올려서 현재 가진 돈 544.0$ 중 145.0$를 베팅한다
내가 받은 카드는 ['HK', 'C3']
13 점
딜러의 카드는 ['D10']와 한장의 히든 카드
보이는 점수는 10 점
HIT!
['H3']
HIT!
['D4']
player: [('H', 'K'), ('C', '3'), ('H', '3'), ('D', '4')]
20
dealer: [('D', '10'), ('D', '8')]
18
이겼다!
arousal:0.989  pleasure0.758

4 번째 게임
베팅 금액을 올려서 현재 가진 돈 689.0$ 중 678.0$를 베팅한다
내가 받은 카드는 ['DQ', 'H2']
12 점
딜러의 카드는 ['CQ', 'C6']와 한장의 히든 카드
보이는 점수는 16 점
HIT!
['S4']
HIT!
['S3']
player: [('D', 'Q'), ('H', '2'), ('S', '4'), ('S', '3')]
19
dealer: [('

In [183]:
last_budget = [x.player.budget_path[-1] for x in game_cand]
before_last_budget = [x.player.budget_path[-2] for x in game_cand]

In [187]:
np.argmax(last_budget)

105

In [186]:
np.max(last_budget)

2199.0

In [505]:
'''
plt.figure(figsize=(20,36))
idx = 0
cnt = 0
for bj in game_path[0]:
    plt.subplot(12,5,idx+1); plt.barh([0,1], [bj.player.character.phenotype['impulsivity'],
                                              bj.player.character.phenotype['stress_reg']]);
    plt.yticks([12,1], ['impulsivity', 'stress_reg']); plt.xlim(0,1); plt.xlabel(cnt);
    plt.subplot(12,5,idx+2); plt.plot(bj.player.bet_path); plt.xlabel('bet (last){}'.format(bj.player.bet_path[-1]));
    plt.subplot(12,5,idx+3); plt.plot(bj.player.budget_path); plt.xlabel('budget'); plt.ylim(0, 7000);
    plt.subplot(12,5,idx+4); plt.plot(bj.player.arousal_path); plt.xlabel('arousal'); plt.ylim(-1, 1);
    plt.subplot(12,5,idx+5); plt.plot(bj.player.pleasure_path); plt.xlabel('pleasure'); plt.ylim(-1, 1);
    plt.tight_layout()
    idx += 5
    cnt += 1
plt.show()
'''

"\nplt.figure(figsize=(20,36))\nidx = 0\ncnt = 0\nfor bj in game_path[0]:\n    plt.subplot(12,5,idx+1); plt.barh([0,1], [bj.player.character.phenotype['impulsivity'],\n                                              bj.player.character.phenotype['stress_reg']]);\n    plt.yticks([12,1], ['impulsivity', 'stress_reg']); plt.xlim(0,1); plt.xlabel(cnt);\n    plt.subplot(12,5,idx+2); plt.plot(bj.player.bet_path); plt.xlabel('bet (last){}'.format(bj.player.bet_path[-1]));\n    plt.subplot(12,5,idx+3); plt.plot(bj.player.budget_path); plt.xlabel('budget'); plt.ylim(0, 7000);\n    plt.subplot(12,5,idx+4); plt.plot(bj.player.arousal_path); plt.xlabel('arousal'); plt.ylim(-1, 1);\n    plt.subplot(12,5,idx+5); plt.plot(bj.player.pleasure_path); plt.xlabel('pleasure'); plt.ylim(-1, 1);\n    plt.tight_layout()\n    idx += 5\n    cnt += 1\nplt.show()\n"